# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
city_data_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Pangnirtung,100,CA,1578395286,99,66.15,-65.72,-40.79,5.95
1,Hithadhoo,100,MV,1578395286,76,-0.60,73.08,84.25,16.44
2,Luderitz,0,NaN,1578395286,44,-26.65,15.16,80.60,13.87
3,Mataura,78,NZ,1578395286,78,-46.19,168.86,48.99,5.99
4,Helong,15,CN,1578395286,73,25.94,116.13,55.85,4.94


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# Store latitude and longitude in locations
locations = city_data_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = city_data_df["Humidity"]

# Plot Heatmap
fig = gmaps.Map()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add layer
fig.add_layer(heat_layer)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_city_data_df = city_data_df.loc[city_data_df['Max Temp'] < 80, :]
ideal_city_data_df = ideal_city_data_df.loc[ideal_city_data_df['Max Temp'] > 70, :]
ideal_city_data_df = ideal_city_data_df.loc[ideal_city_data_df['Wind Speed'] < 10, :]
ideal_city_data_df = ideal_city_data_df.loc[ideal_city_data_df['Cloudiness'] == 0, :]
ideal_city_data_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
35,Pochutla,0,MX,1578395290,73,15.74,-96.47,77.59,7.61
239,Ponta do Sol,0,BR,1578395316,75,-20.63,-46.00,78.19,0.94
244,Chuy,0,UY,1578395247,48,-33.69,-53.46,73.47,9.40
300,Taoudenni,0,ML,1578395324,10,22.68,-3.98,70.83,9.04
320,Reconquista,0,AR,1578395327,27,-29.14,-59.64,75.81,8.81


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_city_data_df
hotel_df['Hotel Name'] = np.nan

target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

idx = 0
for index, row in hotel_df.iterrows():
    # set up a parameters dictionary
    params = {
        "location": f"{row['Lat']}, {row['Lng']}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    if len(results) > 0:
        name = response_json['results'][0]['name']
        hotel_df.iloc[idx, -1] = name
    idx = idx + 1

narrowed_city_df = hotel_df

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

NameError: name 'hotel_info' is not defined